In [ ]:
%matplotlib inline
from os.path import expanduser, join
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

from sklearn.externals import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

from pyrallel.ensemble import EnsembleGrower
from pyrallel.ensemble import sub_ensemble

In [ ]:
from IPython.parallel import Client
lb_view = Client().load_balanced_view()
len(lb_view)

## Loading the dataset

This is a NumPy array version of Fold1 of the [MSLR-WEB10K](http://research.microsoft.com/en-us/projects/mslr/) dataset.

In [ ]:
%%time

data = np.load(expanduser('/data/khodadaa/mslr/MSLR-WEB10K/mslr-web10kFold1.npz'))
X_train, y_train, qid_train = data['X_train'], data['y_train'], data['qid_train']
X_vali, y_vali, qid_vali = data['X_vali'], data['y_vali'], data['qid_vali']
X_test, y_test, qid_test = data['X_test'], data['y_test'], data['qid_test']

Total size in bytes, total number of search results and number of queries:

In [ ]:
(X_train.nbytes + X_vali.nbytes + X_test.nbytes) / 1e6

In [ ]:
len(X_train) + len(X_vali) + len(X_test)

In [ ]:
len(np.unique(qid_train)) + len(np.unique(qid_vali)) + len(np.unique(qid_test))

Concatenate the training and validation sets as a big development set.

In [ ]:
X_dev = np.vstack([X_train, X_vali])
y_dev = np.concatenate([y_train, y_vali])
qid_dev = np.concatenate([qid_train, qid_vali])

In [ ]:
X_dev.shape

In [ ]:
X_dev.dtype

Extract a subset of 500 queries to speed up the learning when prototyping

In [ ]:
def subsample(X, y, qid, size, seed=None):
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid_train, unique_qid[qid_mask])
    return X[subset_mask], y[subset_mask], qid[subset_mask]


X_train_small, y_train_small, qid_train_small = subsample(
    X_train, y_train, qid_train, 500, seed=0)

In [ ]:
X_train_small.shape

Sanity check:

In [ ]:
len(np.unique(qid_train_small))

In [ ]:
X_train_medium, y_train_medium, qid_train_medium = subsample(
    X_train, y_train, qid_train, 1000, seed=0)

In [ ]:
def balance_irrelevant(X, y, qid, seed=None):
    """Subsample the zero-scored entries"""
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    final_mask = np.ones(shape=y.shape, dtype=np.bool)
    for this_qid in unique_qid:
        this_mask = qid == this_qid
        this_y = y[this_mask]
        relevant = this_y >= 2
        ratio = float(np.mean(relevant))
        if ratio > 0.5:
            # already balanced
            continue
            
        final_mask[this_mask] = np.logical_or(
            relevant, np.random.random(len(this_y)) > 0.7) 
    return X[final_mask], y[final_mask], qid[final_mask]

X_balanced_small, y_balanced_small, qid_balanced_small = balance_irrelevant(
    X_train_small, y_train_small, qid_train_small)

In [ ]:
print(len(y_train_small))
print(len(y_balanced_small))

## Quantifying ranking success with NDCG

In [ ]:
def dcg(relevances, rank=10):
    """Discounted cumulative gain at rank (DCG)"""
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)
 
 
def ndcg(relevances, rank=10):
    """Normalized discounted cumulative gain (NDGC)"""
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg

In [ ]:
ndcg([2, 4, 0, 1, 1, 0, 0], rank=5)

In [ ]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=5)

In [ ]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=3)

In [ ]:
ndcg([4, 2, 1, 1, 0, 0, 0], rank=5)

In [ ]:
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    # assume query_ids are sorted
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
    return np.mean(ndcg_scores)


mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

In [ ]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))

In [ ]:
def plot_ndcg_by_trees(model, X, y, qid, rank=10):
    max_n_trees = len(model.estimators_)
    scores = []
    
    if hasattr(model, 'staged_predict'):
        # stage-wise score computation for boosted ensembles
        n_trees = np.arange(max_n_trees) + 1
        for y_predicted in model.staged_predict(X):
            scores.append(mean_ndcg(y, y_predicted, qid, rank=10))
    else:
        # assume forest-type of tree ensemble: use a log scale to speedup
        # the computation
        # XXX: partial predictions could be reused
        n_trees = np.logspace(0, np.log10(max_n_trees), 10).astype(int)
        for j, n in enumerate(n_trees):
            y_predicted = sub_ensemble(model, n).predict(X)
            scores.append(mean_ndcg(y, y_predicted, qid, rank=rank))
            
    plt.plot(n_trees, scores)
    plt.xlabel("Number of trees")
    plt.ylabel("Average NDCG@%d" % rank)
    _ = plt.title("Impact of the number of trees")

In [ ]:
%%time

from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
etr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(etr, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(etr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
rfr.fit(X_balanced_small, y_balanced_small)

In [ ]:
print_evaluation(rfr, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(rfr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor

rfr2 = RandomForestRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
rfr2.fit(X_train_small, y_train_small)

In [ ]:
#rfr.set_params(n_jobs=-1)
print_evaluation(rfr2, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(rfr2, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, random_state=1, verbose=1)
gbr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(gbr, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(gbr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gbr2 = GradientBoostingRegressor(n_estimators=300, max_depth=3,
                                 learning_rate=0.1, loss='ls',
                                 random_state=1, verbose=1)
gbr2.fit(X_dev, y_dev)

In [ ]:
print_evaluation(gbr2, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(gbr2, X_test, y_test, qid_test)

In [ ]:
print_evaluation(gbr2, X_dev, y_dev, qid_dev)

In [ ]:
%%time

from sklearn.ensemble import LambdaMART

lmart= LambdaMART(n_estimators=300, max_depth=3,
                  learning_rate=0.1, random_state=1, verbose=1)
lmart.fit(X_train_small, y_train_small, group=qid_train_small)

In [ ]:
print_evaluation(lmart, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(lmart, X_test, y_test, qid_test)

In [ ]:
print_evaluation(lmart, X_train_small, y_train_small, qid_train_small)

In [ ]:
%%time

lmart= LambdaMART(n_estimators=300, max_depth=3,
                  learning_rate=0.1, random_state=1, verbose=1)
lmart.fit(X_dev, y_dev, group=qid_dev)

In [ ]:
print_evaluation(lmart, X_test, y_test, qid_test)

In [ ]:
print_evaluation(lmart, X_dev, y_dev, qid_dev)

In [ ]:
plot_ndcg_by_trees(lmart, X_test, y_test, qid_test)

## Growing Randomized Trees to predict relevance scores

In [ ]:
from sklearn.ensemble import RandomForestRegressor

base_estimator = RandomForestRegressor(n_estimators=1, min_samples_split=10)
grower = EnsembleGrower(lb_view, base_estimator)

In [ ]:
grower.launch(X_dev, y_dev, n_estimators=200, folder="web10k",
              dump_models=False)

In [ ]:
grower

In [ ]:
#grower.wait()

In [ ]:
%%time

rfr = grower.aggregate_model()
print("Number of trees: {}".format(len(rfr.estimators_)))

In [ ]:
print_evaluation(rfr, X_test, y_test, qid_test)

In [ ]:
plot_ndcg_by_trees(rfr, X_test, y_test, qid_test)

Evaluation of the overfitting of the ensemble:

In [ ]:
print_evaluation(rfr, X_train_medium, y_train_medium, qid_train_medium)

## Comparing with a baseline linear regression models (with different optimizers and input scaling)

In [ ]:
%time lr = LinearRegression().fit(X_dev, y_dev)

In [ ]:
%time y_test_lr = lr.predict(X_test)

In [ ]:
print_evaluation(lr, X_test, y_test, qid_test)

Evaluate overfitting by comparing with training set:

In [ ]:
print_evaluation(lr, X_dev, y_dev, qid_dev)

Interestingly enough, a slight overfitting of the training set from a regression standpoint (higher r2 score) does not seem to cause overfitting from a ranking standpoint. This would have to be checked with cross-validation though.

Let's evaluate the impact of imput feature normalization:

In [ ]:
%%time

nlr = LinearRegression(normalize=True).fit(X_dev, y_dev)

In [ ]:
print_evaluation(nlr, X_test, y_test, qid_test)

In [ ]:
%%time

nlr = LinearRegression(normalize=True).fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(nlr, X_test, y_test, qid_test)

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_small_scaled = scaler.fit_transform(X_train_small)
X_test_scaled = scaler.transform(X_test)

In [ ]:
%%time

sgdlr = SGDRegressor(alpha=1e-7, learning_rate='constant', eta0=1e-5, n_iter=50).fit(X_train_small_scaled, y_train_small)

In [ ]:
print_evaluation(sgdlr, X_test_scaled, y_test, qid_test)

## Comparing with a classification to NDCG ranking reduction models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

In [ ]:
def proba_to_relevance(probas):
    """MCRank-like reduction of classification proba to DCG predictions"""
    rel = np.zeros(probas.shape[0], dtype=np.float32)
    for i in range(probas.shape[1]):
        rel += i * probas[:, i]
    return rel
        
        
class ClassificationRanker(RegressorMixin):
    
    def __init__(self, base_estimator=None):
        self.base_estimator = base_estimator
        
    def fit(self, X, y):
        self.estimator_ = clone(self.base_estimator)
        self.scaler_ = StandardScaler()
        X = self.scaler_.fit_transform(X)
        self.estimator_.fit(X, y)
        
    def predict(self, X):
        X_scaled = self.scaler_.transform(X)
        probas = self.estimator_.predict_proba(X_scaled)
        return proba_to_relevance(probas)

In [ ]:
%%time

logr = ClassificationRanker(LogisticRegression(C=1000))
logr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(logr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.linear_model import SGDClassifier

sgdlogr = SGDClassifier(loss='modified_huber', alpha=1e-8, n_iter=200, learning_rate='constant', eta0=1e-6, n_jobs=-1)
sgdlogrr = ClassificationRanker(sgdlogr)
sgdlogrr.fit(X_train_small_scaled, y_train_small)

In [ ]:
print_evaluation(sgdlogrr, X_test_scaled, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200, max_features=20, min_samples_split=5,
                            random_state=1, n_jobs=-1)
rfr = ClassificationRanker(rfc)
rfr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(rfr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, random_state=1)
gbr = ClassificationRanker(gbc)
gbr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(gbr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import ExtraTreesClassifier

etc = ClassificationRanker(ExtraTreesClassifier(n_estimators=200, random_state=1, n_jobs=-1))
etc.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(etc, X_test, y_test, qid_test)

## Introspecting the distribution of relevance scores predictions

In [ ]:
subset = np.random.permutation(y_test.shape[0])[:10000]

In [ ]:
plt.title('Extra Trees predictions')
plt.scatter(y_test[subset], y_test_etr[subset], alpha=0.1, s=100)
plt.xlabel('True relevance')
plt.ylabel('Predicted relevance')
plt.ylim(-2, 5)
plt.xlim(-2, 5)

In [ ]:
plt.title('Linear Regression predictions')
plt.scatter(y_test[subset], y_test_lr[subset], alpha=0.1, s=100)
plt.xlabel('True relevance')
plt.ylabel('Predicted relevance')
plt.ylim(-2, 5)
plt.xlim(-2, 5)

In [ ]:
plt.hist(y_test, bins=5, alpha=.3, color='b', label='True relevance')
plt.hist(y_test_etr, bins=5, alpha=.3, color='g', label='ET predicted relevance')
plt.legend(loc='best')

For each query, count the number of results with rank 0, 1, 2, 3 or 4.

In [ ]:
unique_qids_test = np.unique(qid_test)
for qid in unique_qids_test[:5]:
    qids = y_test[qid_test == qid].astype(np.int)
    print(np.bincount(qids, minlength=5))